In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import matplotlib
import urllib.request
import time

import radarkit
import algorithms
import chart
import data

In [ ]:
# Read in data files
filename = data.file()
sweep = radarkit.read(filename)
z = sweep['products']['Z']['data']
d = sweep['products']['D']['data']
p = sweep['products']['P']['data']

# Some other parameters from the sweep data
gateSpacing = 1.0e-3 * sweep['gateSizeMeters']
downSamplingRatio = int(sweep['gateSizeMeters'] / 30.0)

## Calibration / Data Extraction

In [ ]:
# Undo range correction and the ZCal in DSP to estimate the SNR
rr = 1.0e-3 * (np.arange(sweep['gateCount']) + 0.5) * sweep['gateSizeMeters']
snr = z - 20.0 * np.log10(rr) + 23

# Transition gate at 10.5 km at the given spacing
g = int((69.0e-6 * 3.0e8 * 0.5) / 30.0) + 5
g = int((g + downSamplingRatio - 1) / downSamplingRatio)

print(g)

z_off = 10.0 * np.log10(1.5 / 67)
snr[:, :g] = snr[:, :g] + z_off

# PhiDP calibration so that the transition is smooth and PhiDP starts ~ 0 deg
p[:, :g] = p[:, :g] + 40;
p[:, g:] = p[:, g:] + 44;

## Calling the Algorithm Function

In [ ]:
tic = time.time()
zc, dc = algorithms.zphi(z, d, p)
toc = time.time()
print('Elapsed time = {} s'.format(toc - tic))

## Visualization

In [ ]:
# Original range (km) and zimuth (rad) of the data
rr = 1.0e-3 * sweep['range']
aa = sweep['azimuth'] * (np.pi / 180.0)

# Plot using chart.Chart()
_ = chart.Chart(aa, rr, z, style='Z')
_ = chart.Chart(aa, rr, zc, style='Z', title='Corrected Z (dBZ)')
_ = chart.Chart(aa, rr, d, style='D')
_ = chart.Chart(aa, rr, dc, style='D', title='Corrected ZDR (dBZ)')